In [1]:
import json, glob
import osgeo.ogr as ogr
import numpy as np
import rasterio as rio
import rasterio.features as features
import rasterio.warp as warp
from shapely.geometry import shape, asShape, GeometryCollection, MultiPolygon
from shapely.wkb import loads as wkb_loads

In [58]:
imagens = glob.glob('*.tif')
imagens

['mapbiomas-matogrossodosul-2018-corte.tif']

In [59]:
def open_raster(imagens):
    crs_affine_all = []
    img_crs_all = []
#     geom_all = {}
    geom_all = {}
    
    for img in imagens:
        
        img_open = rio.open(img)
        img_arr = img_open.read() # array das img
        crs_affine_all.append(img_open.transform) # pega as coordenadas das img
        img_crs_all.append(img_open.crs) #pega os crs das img
        
        name = img.split(".")[0]
        geom_all[name] = []
        
        for geom, val in features.shapes(img_arr, transform = img_open.transform):

            geom = warp.transform_geom(img_open.crs, 'EPSG:4326', geom, precision=5)
            geom_all[name].append(geom)

    return img, img_arr, crs_affine_all, img_crs_all, geom_all

In [60]:
img, img_arr, crs_affine, img_crs, geom_all = open_raster(imagens)

In [64]:
type(geom_all)

dict

In [4]:
#     um jeito alternativo de fazer é:
#     img_shape = features.shapes(img_arr, transform = img.transform)
#     lista= list(img_shape)
#     lista[0][0]
    


In [17]:
# json_dict = rasterio_shp[0]
json_str = json.dumps(geom_all)
ogr_geom = ogr.CreateGeometryFromJson(json_str)
# geom_wkb = wkb_loads(ogr_geom.ExportToWkb())
# geom_wkb

In [22]:
type(ogr_geom)

NoneType

In [ ]:
def export_pol_to_shapefile(unions):
    
    driver_pol = ogr.GetDriverByName("ESRI Shapefile")
    data_source_pol = driver_pol.CreateDataSource("mapbiomas_polygon.shp")
    
    source_pol = osr.SpatialReference()
    source_pol.ImportFromEPSG(32722)

    target_pol = osr.SpatialReference()
    target_pol.ImportFromEPSG(31982)
    
    ogr_transform_pol = osr.CoordinateTransformation(source_pol, target_pol)


    layer = data_source_pol.CreateLayer("estradas_satelite", target_pol, ogr.wkbMultiPolygon)


    field_id_estr = ogr.FieldDefn("id_estr", ogr.OFTInteger)
    layer.CreateField(field_id_estr)
    
    field_bloco_estr = ogr.FieldDefn("bloco",ogr.OFTString)
    field_bloco_estr.SetWidth(20)
    layer.CreateField(field_bloco_estr)

    field_area_estr = ogr.FieldDefn("area", ogr.OFTReal)
    layer.CreateField(field_area_estr)
    
    for count, (union_image, nome_bloco) in enumerate(zip(unions, id_images['bloco_id_intersect'])):
        

        geoms_estradas_wkb = union_image.wkb
        ogr_geoms_estradas = ogr.CreateGeometryFromWkb(geoms_estradas_wkb)
        ogr_geoms_estradas.Transform(ogr_transform_pol)

        area = ogr_geoms_estradas.Area()

        feature = ogr.Feature(layer.GetLayerDefn())
        feature.SetField("id_estr", count+1)
        feature.SetField("bloco", nome_bloco)
        feature.SetField("area", area)
        feature.SetGeometry(ogr_geoms_estradas)
        layer.CreateFeature(feature)
    
    
    return layer.SyncToDisk()


In [ ]:
json_dict = rasterio_shp[0]
json_str = json.dumps(json_dict)
ogr_geom = ogr.CreateGeometryFromJson(json_str)
geom = wkb_loads(ogr_geom.ExportToWkb())


In [14]:
json_dict = rasterio_shp[0]


Affine(0.000269494585915493, 0.0, -51.92190477,
       0.0, -0.0002694945859491756, -20.327976564)